In [1]:
import random
import math
import copy
from heapq import heappush, heappop
from collections import defaultdict
def isSorted(nodes):
    if(len(nodes)==0):
        return True
    for i in range(1,len(nodes)):
        if(nodes[i-1] > nodes[i]):
            return False
    return True

def EuclideanDistance(point1_x, point1_y, point2_x, point2_y):
    internalCalculation = math.pow((point1_x - point2_x),2) + math.pow((point1_y - point2_y),2)
    return math.sqrt(internalCalculation);

def getDistanceBetweenNodes(node, otherNode, AdjHash, Map, DistanceComputation):
    if(node == otherNode):
        return 0;
    if(otherNode in AdjHash and node in AdjHash[otherNode]):
        return AdjHash[otherNode][node]
    return DistanceComputation(Map[node][0], Map[node][1], Map[otherNode][0], Map[otherNode][1])

def computeAdjHashOfCompleteGraph(Map, DistanceComputation):
    # Input::  Map: {0: (0.13037375991631983, 0.17982980099790336), ... }
    nodes = Map.keys()
    assert isSorted(nodes)
    AdjHash= {}
    for node in nodes:
        AdjHash[node] = {}
        for otherNode in nodes:
            AdjHash[node][otherNode] = getDistanceBetweenNodes(node, otherNode, AdjHash, Map, DistanceComputation)
    return AdjHash

def TspGenerator(numberOfCities, lowRange=0.0, highRange=1.0, DistanceComputation=EuclideanDistance):
    Map = {}
    inverseMap = {}
    AdjHash = {}
    for x in range(0, numberOfCities):
        coordinate = (random.uniform(lowRange, highRange), random.uniform(lowRange, highRange))
        tries = 0;
        while(coordinate in inverseMap):
            coordinate = (random.uniform(lowRange, highRange), random.uniform(lowRange, highRange))
            tries+=1;
            if(tries==5):
                print "Unable to Generate Coordinates"
                return ;
        Map[x] = coordinate
        inverseMap[coordinate] = x
    return Map, inverseMap, computeAdjHashOfCompleteGraph(Map, DistanceComputation);

In [2]:
def isAllVisited(VisitedHash, nodes):
    for node in nodes:
        if(node not in VisitedHash):
            return False
    return True

def PrimsAlgorithm(startNode, nodes, AdjHash):
#     print startNode, nodes
    #Input startNode : type int, nodes : type list of int's
    MSTCost = 0;
    h = []
    visitedHash = {}
    visitedHash[startNode] = True
    prevNode = startNode
    MstEdges = []
    while(not isAllVisited(visitedHash, nodes)):
        for node in nodes:
            if(node not in visitedHash.keys()):
#                 print h
                heappush(h, (AdjHash[prevNode][node], (prevNode, node)))
        cost, edge = heappop(h)
        parentNode, minNode = edge
        while(minNode in visitedHash):
            # if min node was alreadyVisited
            cost, edge = heappop(h)
            parentNode, minNode = edge
        MSTCost += cost
        MstEdges.append((parentNode, minNode))
        visitedHash[minNode] = True
    return MSTCost, MstEdges;

def MSTHeuristic(startNode, nodes, AdjHash):
    #Input :: AdjHash is a Adjacency map of the entire set of nodes with the value being the distance
    assert startNode in nodes;
    cost, edges = PrimsAlgorithm(startNode, nodes, AdjHash)
    return cost;
    

    
class TspNode:
    def __init__(self, x, y, nodeNumber, path_cost, parentNode, heuristicCostMap, listOfNodeNumbers, AdjHash, HeuristicFunction=MSTHeuristic):
        self.pos_x = x
        self.pos_y = y
        self.node_number = nodeNumber
        self.g_cost = path_cost
        self.parent = parentNode
        
        # This is not the start node
        if(parentNode!=-1):
            self.nodes_visited = copy.copy(parentNode.nodes_visited)
            MSTSet = frozenset(set(listOfNodeNumbers) - set(self.nodes_visited.keys()))
            if(len(MSTSet)==1):
                heuristicCostMap[MSTSet] = 0;
            if(MSTSet not in heuristicCostMap):
                heuristicCostMap[MSTSet] = MSTHeuristic(self.node_number, list(MSTSet), AdjHash)
            heuristicCost = heuristicCostMap[MSTSet]
        else:
            heuristicCostMap[frozenset(listOfNodeNumbers)] = MSTHeuristic(self.node_number, listOfNodeNumbers, AdjHash)
            heuristicCost = heuristicCostMap[frozenset(listOfNodeNumbers)]
        self.h_cost = heuristicCost
        self.f_cost = self.g_cost + self.h_cost
        
def goalTest(node, listOfNodeNumbers):
    return isAllVisited(node.nodes_visited, list(set(listOfNodeNumbers) - set([node.node_number])))

def appendToFrontier(newNode, Frontier):
    if(newNode.f_cost in Frontier):
        Frontier[newNode.f_cost].append(newNode)
    else:
        Frontier[newNode.f_cost] = [newNode]

def printPathTraversed(node):
    pathTraversed = []
    while(node.parent!=-1):
        pathTraversed.append(node.node_number)
        node=node.parent
    pathTraversed.append(node.node_number)
    pathTraversed = list(reversed(pathTraversed))
    pathTraversed.append(pathTraversed[0])
    return pathTraversed
    
def popMinNode(Frontier):
    m = min(i for i in Frontier.keys() if len(Frontier[i]) > 0)
    minNode = Frontier[m].pop(0)
    return minNode;
        
def hasTraversedPreviously(newNodeNumber, node):
    if(newNodeNumber in node.nodes_visited):
        return True
    return False

def findBestNode(Frontier):
    bestNode = popMinNode(Frontier)
    while((hasTraversedPreviously(bestNode.node_number, bestNode))):
        bestNode = popMinNode(Frontier)
    return bestNode
        
def checkIfFCostIsGreaterThanBestFinalNode(bestFinalNode, node):
    if(bestFinalNode.f_cost <= node.f_cost):
        return False
    return True

def checkNoOtherBetterNodeToExpand(Frontier, finalNode):
    bestNodeToExpand = findBestNode(Frontier)
    return checkIfFCostIsGreaterThanBestFinalNode(finalNode, bestNodeToExpand)

def appendToFrontier(newNode, Frontier):
    if(newNode.f_cost in Frontier):
        Frontier[newNode.f_cost].append(newNode)
    else:
        Frontier[newNode.f_cost] = [newNode]

def expandNode(Node, Frontier, AdjHash, listOfNodeNumbers, Map, heuristicCostMap):
    Node.nodes_visited[Node.node_number] = True
#     print Frontier
    toExpand = list(set(listOfNodeNumbers) - set(Node.nodes_visited.keys()))
    for nodeNumber in toExpand:
        if(nodeNumber != Node.node_number):
            latitude = Map[nodeNumber][0]
            longitude = Map[nodeNumber][1]
            tspNode = TspNode(latitude, longitude, nodeNumber, Node.g_cost+AdjHash[Node.node_number][nodeNumber], Node, heuristicCostMap, listOfNodeNumbers, AdjHash)
            appendToFrontier(tspNode, Frontier)
#     print Frontier
    return ;
    
def solveAstar(startNode, AdjHash, listOfNodeNumbers, Map):
    heuristicCostMap = {}
    #Input startNode : type int, listOfNodeNumbers : type list of int's
    latitude = Map[startNode][0]
    longitude = Map[startNode][1]
    sNode = TspNode(latitude, longitude, startNode, 0, -1, heuristicCostMap, listOfNodeNumbers, AdjHash)
    sNode.nodes_visited = {}
    Frontier = defaultdict(list)
    Frontier[sNode.f_cost] = [sNode]
#     print Frontier
    isCompleted = False
    bestFinalNode = None
    FoundCostToFinal = False;
    noOfNodesExpanded = 0
    while(len(Frontier)!=0 or isCompleted):
        bestNode = findBestNode(Frontier)
        isNodeGoalState = goalTest(bestNode, listOfNodeNumbers)
#         if(isNodeGoalState and checkNoOtherBetterNodeToExpand(Frontier, bestNode)):
#             isCompleted = True
#             break
        if(bestFinalNode != None and bestNode.f_cost > bestFinalNode.g_cost + AdjHash[bestFinalNode.node_number][sNode.node_number]):
            isCompleted = True
            break
        if(isNodeGoalState):
            print " ******* reached Goal State ******* "
            print " Nodes Reached by Goal State of node number %d is %s" %(bestNode.node_number, bestNode.nodes_visited)
            print " ******* ****************** ******* "
            
#             if(bestFinalNode != None):
                #print bestFinalNode.f_cost
                #print bestFinalNode.g_cost + AdjHash[bestFinalNode.node_number][sNode.node_number]
            if(not FoundCostToFinal):
                bestFinalNode = bestNode
                FoundCostToFinal = True
            else:
                if(bestFinalNode.g_cost + AdjHash[bestFinalNode.node_number][sNode.node_number] > bestNode.g_cost  + AdjHash[bestNode.node_number][sNode.node_number]):
                    bestFinalNode=bestNode
        if(not isNodeGoalState):
            noOfNodesExpanded+=1
            print "Expanding Node : %d and nodes visited %s" %(bestNode.node_number, bestNode.nodes_visited)
            expandNode(bestNode, Frontier, AdjHash, listOfNodeNumbers, Map, heuristicCostMap)
    print heuristicCostMap
    totalCost = bestFinalNode.g_cost + AdjHash[bestFinalNode.node_number][sNode.node_number]
    return bestFinalNode, totalCost, noOfNodesExpanded

In [3]:
Map, inverseMap, AdjHash = TspGenerator(5)
nodes = Map.keys()

In [4]:
startNode = 0
bestFinalNode, TotalCost, noOfNodesExpanded = solveAstar(startNode, AdjHash, nodes, Map)

Expanding Node : 0 and nodes visited {}
Expanding Node : 4 and nodes visited {0: True}
Expanding Node : 2 and nodes visited {0: True, 4: True}
Expanding Node : 1 and nodes visited {0: True, 2: True, 4: True}
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 3 is {0: True, 1: True, 2: True, 4: True}
 ******* ****************** ******* 
Expanding Node : 1 and nodes visited {0: True, 4: True}
Expanding Node : 2 and nodes visited {0: True, 1: True, 4: True}
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 3 is {0: True, 1: True, 2: True, 4: True}
 ******* ****************** ******* 
Expanding Node : 2 and nodes visited {0: True}
Expanding Node : 1 and nodes visited {0: True, 2: True}
Expanding Node : 4 and nodes visited {0: True, 1: True, 2: True}
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 3 is {0: True, 1: True, 2: True, 4: True}
 ******* ****************** ******* 
Expanding Node : 4 and 

In [60]:
printPathTraversed(bestFinalNode)
print TotalCost

1.37203170194


In [6]:
# Testing Prims Algorithm
ExamplestartNode = 1
Examplenodes = [1,2,3,4]
ExampleAdjHash =  {
    1:{ 
        1: 0,
        2: 10,
        3: 15,
        4: 20
      },
    2:{ 
        1: 10,
        2: 0,
        3: 35,
        4: 25
      },
    3:{ 
        1: 15,
        2: 35,
        3: 0,
        4: 30
      },
    4:{ 
        1: 20,
        2: 25,
        3: 30,
        4: 0
      }
}
ExampleMap = {
    1:(0,0),
    2:(1,1),
    3:(2,2),
    4:(3,3)
}
ExampleheuristicCost, Edges = PrimsAlgorithm(ExamplestartNode, Examplenodes, ExampleAdjHash)


In [7]:
ExamplebestNode, ExampleTotalCost, ExamplenoOfNodesExpanded = solveAstar(ExamplestartNode, ExampleAdjHash, Examplenodes, ExampleMap)

Expanding Node : 1 and nodes visited {}
Expanding Node : 2 and nodes visited {1: True}
Expanding Node : 4 and nodes visited {1: True, 2: True}
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 3 is {1: True, 2: True, 4: True}
 ******* ****************** ******* 
Expanding Node : 3 and nodes visited {1: True}
Expanding Node : 4 and nodes visited {1: True, 3: True}
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 2 is {1: True, 3: True, 4: True}
 ******* ****************** ******* 
Expanding Node : 3 and nodes visited {1: True, 2: True}
Expanding Node : 2 and nodes visited {1: True, 3: True}
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 4 is {1: True, 2: True, 3: True}
 ******* ****************** ******* 
 ******* reached Goal State ******* 
 Nodes Reached by Goal State of node number 4 is {1: True, 2: True, 3: True}
 ******* ****************** ******* 
Expanding Node : 4 and nodes visited {

In [72]:
print printPathTraversed(ExamplebestNode)
print ExampleTotalCost

[1, 2, 4, 3, 1]
80


In [16]:
def getInputFromFile(fileName):
    try:
        Map = {}
        inverseMap = {}
        in_file = open(fileName, 'r')
        lines = in_file.readlines()
        ind = 0;
        for line in lines:
            if(line[0]=='1'):
                ind+=1
        for line in lines[7:len(lines)-1]:
            data = line.split()
            nodeNo = int(data[0])
            latitude, longitude = float(data[1]), float(data[2])
            coordinate = (latitude, longitude)
            Map[nodeNo] = coordinate
            inverseMap[coordinate] = nodeNo
    finally:
        in_file.close()
    return Map, inverseMap, computeAdjHashOfCompleteGraph(Map, DistanceComputation=EuclideanDistance)


#GEOM-norm
M_PI = 3.14159265358979323846264

def geom_edgelen(xi, xj, yi, yj):
    lati = M_PI * xi / 180.0;
    latj = M_PI * xj / 180.0;

    longi = M_PI * yi / 180.0;
    longj = M_PI * yj / 180.0;

    q1 = math.cos (latj) * math.sin(longi - longj);
    q3 = math.sin((longi - longj)/2.0);
    q4 = math.cos((longi - longj)/2.0);
    q2 = math.sin(lati + latj) * q3 * q3 - math.sin(lati - latj) * q4 * q4;
    q5 = math.cos(lati - latj) * q4 * q4 - math.cos(lati + latj) * q3 * q3;
    return (int) (6378388.0 * math.atan2(sqrt(q1*q1 + q2*q2), q5) + 1.0);



In [18]:
# Optional Question Answers : (Q6.4)
WCityMap, WCityinverseMap, WCityAdjHash = getInputFromFile('wi29.tsp.txt')
WCitystartNode = 1
WCitynodes = WCityMap.keys()
print WCityMap

{1: (20833.3333, 17100.0), 2: (20900.0, 17066.6667), 3: (21300.0, 13016.6667), 4: (21600.0, 14150.0), 5: (21600.0, 14966.6667), 6: (21600.0, 16500.0), 7: (22183.3333, 13133.3333), 8: (22583.3333, 14300.0), 9: (22683.3333, 12716.6667), 10: (23616.6667, 15866.6667), 11: (23700.0, 15933.3333), 12: (23883.3333, 14533.3333), 13: (24166.6667, 13250.0), 14: (25149.1667, 12365.8333), 15: (26133.3333, 14500.0), 16: (26150.0, 10550.0), 17: (26283.3333, 12766.6667), 18: (26433.3333, 13433.3333), 19: (26550.0, 13850.0), 20: (26733.3333, 11683.3333), 21: (27026.1111, 13051.9444), 22: (27096.1111, 13415.8333), 23: (27153.6111, 13203.3333), 24: (27166.6667, 9833.3333), 25: (27233.3333, 10450.0), 26: (27233.3333, 11783.3333), 27: (27266.6667, 10383.3333), 28: (27433.3333, 12400.0), 29: (27462.5, 12992.2222)}


In [19]:
WCitybestNode, WCityTotalCost, WCitynoOfNodesExpanded = solveAstar(WCitystartNode, WCityAdjHash, WCitynodes, WCityMap)

Expanding Node : 1 and nodes visited {}
Expanding Node : 2 and nodes visited {1: True}
Expanding Node : 6 and nodes visited {1: True, 2: True}
Expanding Node : 5 and nodes visited {1: True, 2: True, 6: True}
Expanding Node : 4 and nodes visited {1: True, 2: True, 5: True, 6: True}
Expanding Node : 8 and nodes visited {1: True, 2: True, 4: True, 5: True, 6: True}
Expanding Node : 7 and nodes visited {1: True, 2: True, 4: True, 5: True, 6: True, 8: True}
Expanding Node : 9 and nodes visited {1: True, 2: True, 4: True, 5: True, 6: True, 7: True, 8: True}
Expanding Node : 3 and nodes visited {1: True, 2: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True}
Expanding Node : 13 and nodes visited {1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True}
Expanding Node : 12 and nodes visited {1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True, 13: True}
Expanding Node : 10 and nodes visited {1: True, 2: True, 3: True, 4: True, 5: True

In [21]:
printPathTraversed(WCitybestNode)

[1,
 2,
 6,
 5,
 4,
 8,
 7,
 9,
 3,
 13,
 12,
 10,
 11,
 15,
 19,
 18,
 22,
 23,
 21,
 29,
 28,
 26,
 20,
 16,
 24,
 27,
 25,
 17,
 14,
 1]

Q 6.6
Implement and test a hill-climbing method to solve TSPs. Compare the results with the optimal solutions obtained using A*


In [76]:
#http://homepage.divms.uiowa.edu/~hzhang/c231/ch12.pdf
#https://www.cs.cmu.edu/afs/cs/academic/class/15381-s07/www/slides/012507searchlocal.pdf
class HillNode:
    def __init__(self, x, y, nodeNumber, path_cost, parentNode, AdjHash):
        self.pos_x = x
        self.pos_y = y
        self.node_number = nodeNumber
        self.g_cost = path_cost
        self.parent = parentNode

def CostTour(Tour, AdjHash):
    prev = Tour[0].node_number
    Cost = 0.0
    for ind in range(1,len(Tour)):
        node = Tour[ind]
        Cost+=AdjHash[prev][node.node_number]
        prev = node.node_number
    #add Cost For Start as well
    Cost+=AdjHash[prev][0]
    return Cost
       
def printTour(Tour):
    Traversal = []
    for ind in range(0,len(Tour)):
        node = Tour[ind]
        Traversal.append(node.node_number)
    Traversal.append(Tour[0].node_number)
    print str(Traversal)
    return str(Traversal)

def findNeighbour(Tour,AdjHash,pathCost):
    prev = 0
    # I pick two nodes randomly except start and swap
    index1=1
    index2=1
    newTour = copy.copy(Tour)
    Cost = 0
    changed = False
    while(index1==index2):
        index1 = random.randint(1, len(Tour)-1)
        index2 = random.randint(1, len(Tour)-1)
    swapNode = newTour[index1]
    newTour[index1] = newTour[index2]
    newTour[index2] = swapNode
    for ind in range(1,len(newTour)):
        node = Tour[ind]
        Cost+=AdjHash[prev][node.node_number]
        node.g_cost = Cost
        prev = node.node_number
    #add Cost For Start as well
    Cost+=AdjHash[prev][0]
    newTourCost = CostTour(newTour,AdjHash)
    oldTourCost = CostTour(Tour,AdjHash)
    print "new Tour %s Traversal has a cost of %f while old Tour %s has a cost of %f" %(printTour(newTour), newTourCost, printTour(Tour), oldTourCost)
    if(newTourCost<oldTourCost):
        changed = True
        Tour = newTour
    return newTour, changed, Cost
        
    
def HillClimbing(startNodeNumber, AdjHash, listOfNodeNumbers, Map):
    Tour = []
    prevNode = -1
    prevNodeNumber = 0
    pathCost = 0.0
    MAX_ITER = 100
    iterations = 0
    for ind in range(0,len(listOfNodeNumbers)):
        nodeNumber = listOfNodeNumbers[ind]
        latitude = Map[nodeNumber][0]
        longitude = Map[nodeNumber][1]
        node = HillNode(latitude, longitude, nodeNumber, pathCost, prevNode, AdjHash)
        if(ind<len(listOfNodeNumbers)-1):
            nextNodeNumber = listOfNodeNumbers[ind+1]
        else:
            nextNodeNumber = listOfNodeNumbers[0]
        pathCost+= AdjHash[nodeNumber][nextNodeNumber];
        Tour.append(node);
        prevNode = node
    changed=True
    print printTour(Tour);
    while(iterations < MAX_ITER):
        newTour, changed, pathCost = findNeighbour(Tour,AdjHash,pathCost)
        if(changed):
            Tour = copy.copy(newTour)
            printTour(Tour)
        if(iterations >= MAX_ITER):
            break
        iterations+=1
    print printTour(Tour)
    print AdjHash
    return pathCost
        
        
        
    
    

In [78]:
startNode = 0
print AdjHash
HillClimbing(startNode, AdjHash, nodes, Map)

{0: {0: 0, 1: 0.37830285851251194, 2: 0.2975342448401061, 3: 0.4553233847033964, 4: 0.1912886513350311}, 1: {0: 0.37830285851251194, 1: 0, 2: 0.15494081373829133, 3: 0.5303928427176278, 4: 0.19213207734525714}, 2: {0: 0.2975342448401061, 1: 0.15494081373829133, 2: 0, 3: 0.37834677481435997, 4: 0.16945203676511278}, 3: {0: 0.4553233847033964, 1: 0.5303928427176278, 2: 0.37834677481435997, 3: 0, 4: 0.4871159036371785}, 4: {0: 0.1912886513350311, 1: 0.19213207734525714, 2: 0.16945203676511278, 3: 0.4871159036371785, 4: 0}}
[0, 1, 2, 3, 4, 0]
[0, 1, 2, 3, 4, 0]
[0, 4, 2, 3, 1, 0]
[0, 1, 2, 3, 4, 0]
new Tour [0, 4, 2, 3, 1, 0] Traversal has a cost of 1.647783 while old Tour [0, 1, 2, 3, 4, 0] has a cost of 1.589995
[0, 1, 4, 3, 2, 0]
[0, 1, 2, 3, 4, 0]
new Tour [0, 1, 4, 3, 2, 0] Traversal has a cost of 1.733432 while old Tour [0, 1, 2, 3, 4, 0] has a cost of 1.589995
[0, 1, 2, 4, 3, 0]
[0, 1, 2, 3, 4, 0]
new Tour [0, 1, 2, 4, 3, 0] Traversal has a cost of 1.645135 while old Tour [0, 1, 2, 

1.372031701936336

Propose an inadmissible heuristic for the TSP problem and compare the performance of A* (number of nodes expanded and solution quality) with the inadmissible heuristic versus the admissible one.


In [ ]:
# An inadmissible Heurisitic could be 2 Times the Minimum Spanning Tree because it is an overestimate of the true cost to reaching the solution
def InadmissibleMSTHeuristic(startNode, nodes, AdjHash):
    return 2*MSTHeuristic(startNode, nodes, AdjHash)
